In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Function to scrape individual product page
def get_product_details(product_url):
    try:
        # Send GET request to the product page
        response = requests.get(product_url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract Seller Feedback Rating
        feedback_rating = None
        feedback_element = soup.find("span", class_="ux-textspans ux-textspans--PSEUDOLINK")
        if feedback_element:
            feedback_rating = feedback_element.get_text(strip=True)

        # Extract Seller Reviews
        reviews = []
        feedbacks = soup.find_all("div", class_="fdbk-container__details__comment")
        for feedback in feedbacks:
            reviews.append(feedback.get_text(strip=True))
        
        return description, feedback_rating, reviews
    
    except Exception as e:
        print(f"Error while scraping {product_url}: {e}")
        return None, None, None


# eBay URL to search for smart bins
search_term = "smart bin"
url = f"https://www.ebay.com/sch/i.html?_nkw={search_term.replace(' ', '+')}&_sacat=0&_from=R40&_trksid=p2334524.m570.l1313&_blrs=spell_auto_correct&_odkw=smartbin&_osacat=0"

# Send GET request to eBay
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# List to store extracted product details
products = []

# Loop through all items on the page and extract the relevant data
for item in soup.find_all("li", class_="s-item"):
    try:
        title = item.find("a", class_="s-item__link").get_text(strip=True)
        title = re.sub(r"Opens in a new window or tab", "", title).strip()
    except AttributeError:
        title = None
    
    try:
        price = item.find("span", class_="s-item__price").get_text(strip=True)
    except AttributeError:
        price = None

    try:
        link = item.find("a", class_="s-item__link")["href"]
    except AttributeError:
        link = None

    try:
        shipping = item.find("span", class_="s-item__shipping").get_text(strip=True)
    except AttributeError:
        shipping = None

    # Scrape additional details from the individual product page
    description, feedback_rating, reviews = get_product_details(link)
    
    # Append each product's details to the list
    products.append({
        'Title': title,
        'Price': price,
        'Shipping': shipping,
        'Link': link,
        'Seller Feedback Rating': feedback_rating,
        'Seller Reviews': reviews
    })

# Convert the list of product details into a DataFrame
df = pd.DataFrame(products)

# Remove rows where the Title is "Shop on eBay"
df = df[df['Title'] != "Shop on eBay"]

# Display the cleaned DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv("ebay_smart_bins_with_details.csv", index=False)
